<a href="https://colab.research.google.com/github/Astribus/Astribus/blob/main/Whitestone_Transit_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Goal:** Has the 2025 Queens Bus route redesign improved reliability?




**Specifics:** The Q15A bus route that ran from Flushing to deep whitestone was removed and replace with the Q61. Has this improve reliability compared to the previous years?

In [25]:
import pandas as pd
import plotly.express as px
# This is the NEW 2025 link for MTA Bus Wait Assessment
# Source: https://data.ny.gov/d/v4z4-2h6n
url = "https://data.ny.gov/api/views/v4z4-2h6n/rows.csv?accessType=DOWNLOAD"

# Load the data
df = pd.read_csv(url)

# Displays the amount of rows we have
print(f"Success! We loaded {len(df)} rows of data.")

# Show the first 5 rows
df.head()


Success! We loaded 14158 rows of data.


,month,borough,day_type,trip_type,route_id,period,number_of_trips_passing_wait,number_of_scheduled_trips,wait_assessment
0,2025-01-01,Bronx,1,LCL/LTD,BX1,Off-Peak,7314,9823,0.744579
1,2025-01-01,Bronx,1,LCL/LTD,BX1,Peak,4054,7054,0.574709
2,2025-01-01,Bronx,1,UNKNOWN,BX1,Off-Peak,2990,4355,0.686567
3,2025-01-01,Bronx,1,UNKNOWN,BX1,Peak,99,122,0.811475
4,2025-01-01,Bronx,2,LCL/LTD,BX1,Off-Peak,830,1147,0.723627


In [26]:
#Updated 2025 NYC Whitestone Metro buses
active_whitestone_2025 = ['Q15', 'Q16', 'Q61', 'QM2', 'QM32']

# Filter data to these bus routes
df_current_whitestone = df[df['route_id'].isin(active_whitestone_2025)].copy()

# Check if we have data for new Q61 specifically
q61_data = df_current_whitestone[df_current_whitestone['route_id'] == 'Q61']

print(f"Total current Whitestone rows: {len(df_current_whitestone)}")
print(f"New Q61 specific rows found: {len(q61_data)}")

#show first few rows of 2025 data
df_current_whitestone.head()

Total current Whitestone rows: 180
New Q61 specific rows found: 24


,month,borough,day_type,trip_type,route_id,period,number_of_trips_passing_wait,number_of_scheduled_trips,wait_assessment
764,2025-01-01,Queens,1,UNKNOWN,Q15,Off-Peak,3368,3988,0.844534
765,2025-01-01,Queens,1,UNKNOWN,Q15,Peak,1637,2307,0.709580
766,2025-01-01,Queens,2,UNKNOWN,Q15,Off-Peak,833,1018,0.818271
767,2025-01-01,Queens,2,UNKNOWN,Q15,Peak,342,464,0.737069
772,2025-01-01,Queens,1,UNKNOWN,Q16,Off-Peak,5860,6627,0.884261


In [27]:
# Calculation of average wait(On-time %) for each bus
reliability_report = df_current_whitestone.groupby('route_id')['wait_assessment'].mean().reset_index()

#Sort from best to worst
reliability_report = reliability_report.sort_values(by='wait_assessment', ascending=False)

print("Whitestone Bus Reliability Rankings (Best to Worst):")
print(reliability_report)

Whitestone Bus Reliability Rankings (Best to Worst):
  route_id  wait_assessment
1      Q16         0.803019
3      QM2         0.789758
0      Q15         0.788386
4     QM32         0.788149
2      Q61         0.753157


In [28]:
# Bar Chart
fig = px.bar(reliability_report,
             x='route_id',
             y='wait_assessment',
             title = '2025 Whitestone Bus Reliability Comparison',
             labels = {'route_id': 'Bus Route', 'wait_assessment': 'On-Time Performance (%)'},
             color = 'wait_assessment',
             color_continuous_scale = 'Viridis'
             )

fig.show()

In [29]:
# Frequency vs Reliability
# Attempt to see if there are enough Q16's running to meet the high demand based on personal experiences
summary = df_current_whitestone.groupby('route_id').agg({
    'wait_assessment': 'mean',
    'route_id': 'count'
}).rename(columns={'route_id': 'service_frequency_score'})

print("Whitestone Strategic Summary: ")
print(summary.sort_values(by='wait_assessment', ascending=False))


Whitestone Strategic Summary: 
          wait_assessment  service_frequency_score
route_id                                          
Q16              0.803019                       44
QM2              0.789758                       44
Q15              0.788386                       44
QM32             0.788149                       24
Q61              0.753157                       24


**Theory:** Areas with higher car ownership and wealth to maintain expenses have lower bus reliability/priority

In [30]:
# Month to month reliability flucuations
volatility = df_current_whitestone.groupby('route_id')['wait_assessment'].std().reset_index()
volatility.columns = ['route_id', 'reliability_volatility']

# Merge with summary
top_notch_stats = summary.merge(volatility, on = 'route_id')

print("Advanced Whitestone Analytics: ")
print(top_notch_stats.sort_values(by = 'reliability_volatility', ascending = False))

Advanced Whitestone Analytics: 
  route_id  wait_assessment  service_frequency_score  reliability_volatility
4     QM32         0.788149                       24                0.115376
3      QM2         0.789758                       44                0.069461
2      Q61         0.753157                       24                0.066390
1      Q16         0.803019                       44                0.057429
0      Q15         0.788386                       44                0.047250


QM32 is chaotic. Some months it's great, and some months it's a disaster

In [32]:
# calculate correlation between frequency and volatility
# We want to see if buses that run less often are more "chaotic"
correlation_value = top_notch_stats['service_frequency_score'].corr(top_notch_stats['reliability_volatility'])

print(f"Correlation between Frequency and Volatility: {correlation_value:.2f}")

#visualization
fig = px.scatter(top_notch_stats,
                 x = 'service_frequency_score',
                 y = 'reliability_volatility',
                 text = 'route_id',
                 title = 'Whitestone Transit: Frequency vs Volatility',
                 labels = {'service_frequency_score': 'Buses per Month (Frequency)',
                           'reliability_volatility': 'Consistency (Volatility)'},
                 trendline = 'ols')

fig.show()

Correlation between Frequency and Volatility: -0.69


**Analysis of Chart:**

QM32 (Top Left): Outlier. Lowest frequency and highest volatility. This proves that express service in Whitestone is currently the most "unstable" option for commuters

---
Q15 and Q16 (Bottom right): High frequency leads to more consistent, predictable services

---

Q61 (Bottom left): Low frequency like QM32 but more consistent.
This **supports** my theory that low ridership in deep Whitestone keeps the bus on time because it's not spending 5 minutes at every stop boarding 50 people.



**Final Project Findings: The Whitestone Transit Reliability Gap**

---


1) **Reliability Leader:** The Q16 is the most reliable route (80.3%), likely due to its high frequency and "Main Street" priority.

2) **The Redesign Lag:** The new Q61 is the least reliable local route (75.3%), suggesting that the 2025 redesign has not yet optimized this "deep" neighborhood link.

3) **The Volatility Peak:** The QM32 exhibits the highest volatility (0.115), making it a "high-risk" route for commuters despite its express status.

Statistical Conclusion: There is a **-0.69** correlation between service frequency and reliability, suggesting that simply adding more buses may/may not solve the consistency issues without addressing traffic bottlenecks at the Whitestone Bridge.